In [1]:
import pandas as pd
import scanpy as sc
from pathlib import Path



# Configuration
data_dir = Path('../../../data/')
h5ad = data_dir / 'gex/bcells.h5ad.gz'
restrict_to_TBd6 = False
compute_sharing = True  # Flag to control the sharing computation
vdj_infile = data_dir / 'vdj/integrated_cell_calls_ambient_annotated.tsv.gz'
sharing_outfile = data_dir / 'annotation/sharing_labels_gex.tsv.gz'

# Ensure output directory exists
output_dir = sharing_outfile.parent
output_dir.mkdir(parents=True, exist_ok=True)
seq_identifier = 'vdj_sequence'

# Mapping tissue combinations to DataFrame columns
tissue_combo_to_column = {
    'LN_LN': 'shared_LN_LN',
    'LN_SP': 'shared_LN_SP',
    'LN_PB': 'shared_LN_PB',
    'BM_PB': 'shared_BM_PB',
    'SP_PB': 'shared_SP_PB',
    'SP_BM': 'shared_SP_BM',
    'LN_BM': 'shared_LN_BM',
    # Add more mappings as needed
}

adata = sc.read_h5ad(h5ad, backed='r')

# subset to donor 6, the only one with multiple lymph nodes
if restrict_to_TBd6:
    adata = adata[adata.obs.donor == 'TBd6']
adata = adata.to_memory()
# map whether the IGHC is switched
adata.obs.loc[:,"switched"] = adata.obs.c_call.map(IGH_switched())
# remove GEX profiles without VDJ sequences
adata = adata[adata.obs.vdj_sequence != 'nan']
# use only a subset of the data that is relevant for merge
adata.obs = adata.obs[['cb', 'vdj_sequence', 'celltypist', 'sample_uid']]

# Function to check if two specific locations are shared within a group
def shared_locations(group, subanatomical, loc1, loc2):
    if group.empty:
        return pd.NA  # Return pd.NA if the group is empty
    if not subanatomical:
        return (loc1 in group['tissue'].values) and (loc2 in group['tissue'].values)
    else:
        return (loc1 in group['subanatomical_location'].values) and (loc2 in group['subanatomical_location'].values)

if compute_sharing:
    # Load the data with specified dtypes
    df = pd.read_table(vdj_infile, index_col=0, usecols=["vdj_sequence", "lineage_id", 'v_mismatch', 'donor', 'probable_hq_single_b_cell', 'subanatomical_location', 'tissue', 'locus', 'c_call', 'sample_uid', 'cb'])

    # Drop rows with missing 'locus'
    df = df.dropna(subset=['locus'])

    # Subset to LNs
    df_LN = df[df.tissue == 'LN']

    # Print value counts for subanatomical locations within LNs
    print(df_LN.subanatomical_location.value_counts())

    # Identify sequences shared between LNs
    shared_LNs = df_LN.groupby(seq_identifier)['subanatomical_location'].nunique() > 1
    df['shared_LN_LN'] = df[seq_identifier].map(shared_LNs)

    # Identify sequences found in a single LN
    found_single_LN = df_LN.groupby(seq_identifier)['subanatomical_location'].nunique() == 1
    df['found_in_single_LN'] = df[seq_identifier].map(found_single_LN)

    # Dictionary to hold shared categories
    shared_categories = {
        'shared_LN_SP': {'subanatomical': False, 'loc1': 'LN', 'loc2': 'SP'},
        'shared_SP_PB': {'subanatomical': False, 'loc1': 'SP', 'loc2': 'PB'},
        'shared_BM_PB': {'subanatomical': False, 'loc1': 'BM', 'loc2': 'PB'},
        'shared_LN_PB': {'subanatomical': False, 'loc1': 'LN', 'loc2': 'PB'},
        'shared_LN_BM': {'subanatomical': False, 'loc1': 'LN', 'loc2': 'BM'},
        'shared_SP_BM': {'subanatomical': False, 'loc1': 'SP', 'loc2': 'BM'},
    }
    # Iterate through the dictionary to compute shared categories
    for category, params in shared_categories.items():
        shared = df.groupby(seq_identifier).apply(shared_locations, params['subanatomical'], params['loc1'], params['loc2'])
        df[category] = df[seq_identifier].map(shared)

    # Write the DataFrame to a file
    df.reset_index().to_csv(sharing_outfile, sep="\t", index=False)
    
# load output
shared = pd.read_table(sharing_outfile, index_col=0)

print(shared.shape[0], "antibody assemblies to analyze for sharing")
shared = shared[shared.probable_hq_single_b_cell == True]
print(shared.shape[0], "number of hq single B cells transcriptomes to detect GEX differences amongst shared cells")

# Merging the DataFrames with suffixes for common columns
merged_df = pd.merge(adata.obs, shared, left_on=['cb', 'vdj_sequence'], right_on=['cb', 'vdj_sequence'], suffixes=('_left', ''), how='left')

# Dropping the columns duplicated columns for df on right
# Identify columns containing "_right" in their names
cols_to_remove = [col for col in merged_df.columns if '_left' in col]

# Drop the identified columns from the DataFrame
merged_df.drop(columns=cols_to_remove, inplace=True)
merged_df.index = adata.obs.index

# Convert all columns in adata.obs to the appropriate dtype
for col in merged_df.columns:
    if merged_df[col].dtype == 'object':
        merged_df[col] = merged_df[col].astype('category')
    elif pd.api.types.is_float_dtype(merged_df[col]):
        merged_df[col] = merged_df[col].astype('float32')
    elif pd.api.types.is_integer_dtype(merged_df[col]):
        merged_df[col] = merged_df[col].astype('int32')
    else:
        merged_df[col] = merged_df[col].astype('category')

# Create a copy of the AnnData object before modifying
adata = adata.copy()
adata.obs = merged_df
adata = adata[~adata.obs.sample_uid.isna()]

# Now write to H5AD
if restrict_to_TBd6:
    adata.write_h5ad("../../../data/annotation/TBd6_sharing.h5ad.gz", compression='gzip')
else:
    adata.write_h5ad("../../../data/annotation/all_sharing.h5ad.gz", compression="gzip")

/tmp/ipykernel_109895/2500660163.py:83: DtypeWarning: Columns (2,4,5,76,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table(vdj_infile, index_col=0, usecols=["vdj_sequence", "lineage_id", 'v_mismatch', 'donor', 'probable_hq_single_b_cell', 'subanatomical_location', 'tissue', 'locus', 'c_call', 'sample_uid', 'cb'])


SDLN1    146273
SDLN3     56045
SDLN2     46962
MELN1     33967
MELN      26779
Name: subanatomical_location, dtype: int64


/tmp/ipykernel_109895/2500660163.py:120: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  shared = pd.read_table(sharing_outfile, index_col=0)


1160737 antibody assemblies to analyze for sharing
202489 number of hq single B cells transcriptomes to detect GEX differences amongst shared cells


: 